# Imports
Ici on peut retrouver les importations nécéssaires pour le projet
Marche à suivre pour sklearn: https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [2]:
import nltk
# get punkt and perceptron average
nltk.download("averaged_perceptron_tagger")
nltk.download("stopwords")
nltk.download("punkt")
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\WickkaWizz\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\WickkaWizz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WickkaWizz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Feature Extraction
Dans cette section nous allons extraire les features du texte en utilisant nltk pour générer les stop-words que nous allons enlever du texte, en faisant le stemming des données avec Porter, en faisant le POS tagging et pour terminer nous allons aussi établir une grammaire pour pouvoir voir la structure du texte.

## Porter
Ici nous avons le stemmer Porter et nous faisons quelques tests pour vérifier qu'il fonctionne

In [3]:
# https://www.datacamp.com/tutorial/stemming-lemmatization-python
porter = PorterStemmer()
print("Porter Stemmer")
print(porter.stem("cats"))
print(porter.stem("trouble"))
print(porter.stem("troubling"))
print(porter.stem("troubled"))

Porter Stemmer
cat
troubl
troubl
troubl


In [4]:
# https://scikit-learn.org/stable/modules/feature_extraction.html
# https://www.guru99.com/pos-tagging-chunking-nltk.html
# https://www.datacamp.com/tutorial/stemming-lemmatization-python
corpus = []
f = open("data/interest-original.txt", "r")
for x in f:
    corpus.append(x)
# We remove all the $$ from the list to make it cleaner
sentence_corpus = [i for i in corpus if i != "$$\n"]
#print(sentence_corpus[1])
# We tokenize our corpus into words, for each sentence
corpus_tokens = []
for i in sentence_corpus:
    corpus_tokens.append(nltk.word_tokenize(i))
#print(corpus_tokens)
# We remove all the stop-words before stemming the text
# We do a double for loop because we want to keep the structure of sentences and words intact
stop_words = set(stopwords.words("english"))
corpus_stop_words = []
for i in corpus_tokens:
    sw = []
    for e in i:
       if e.casefold() not in stop_words:
           sw.append(e)
    corpus_stop_words.append(sw)
#print(corpus_stop_words)
# We make the stemming of all the tokens
# We do a double for loop because we want to keep the structure of sentences and words intact
corpus_stemmed = []
for i in corpus_stop_words:
    stemming = []
    for e in i:
        stemming.append(porter.stem(e))
    corpus_stemmed.append(stemming)
# We do this list for the vectorizers, because they need a list of sentences
sent_corpus_stemmed = []
for i in corpus_stemmed:
    sent_corpus_stemmed.append(" ".join(i))
#print(corpus_stemmed)
# We make the POS tagging
# corpus_tag = []
# for i in corpus_stemmed:
#     corpus_tag.append(nltk.pos_tag(i))
corpus_tag = nltk.pos_tag_sents(corpus_stemmed)
#print(corpus_tag)
# We apply the grammar on the text, so we can get the structure. We get trees for each sentence
# TODO verify if this grammar is enough for our text
grammar = "NP: {<DT>?<JJ>*<NN>}"
cp = nltk.RegexpParser(grammar)
result = []
for i in corpus_tag:
    result.append(cp.parse(i))
print(result)
result[0].draw()    # It will draw the pattern graphically which can be seen in Noun Phrase chunking

[Tree('S', [Tree('NP', [('yield', 'NN')]), Tree('NP', [('money-market', 'JJ'), ('mutual', 'JJ'), ('fund', 'NN')]), Tree('NP', [('continu', 'NN')]), Tree('NP', [('slide', 'NN')]), (',', ','), ('amid', 'IN'), Tree('NP', [('sign', 'JJ'), ('portfolio', 'NN')]), Tree('NP', [('manag', 'NN')]), ('expect', 'VBP'), Tree('NP', [('declin', 'JJ'), ('interest_6', 'JJ'), ('rate', 'NN')]), ('.', '.')]), Tree('S', [('longer', 'JJR'), Tree('NP', [('matur', 'NN')]), ('thought', 'VBD'), Tree('NP', [('indic', 'JJ'), ('declin', 'JJ'), ('interest_6', 'NN')]), Tree('NP', [('rate', 'NN')]), Tree('NP', [('permit', 'NN')]), Tree('NP', [('portfolio', 'NN')]), Tree('NP', [('manag', 'NN')]), Tree('NP', [('retain', 'NN')]), Tree('NP', [('rel', 'NN')]), ('higher', 'JJR'), Tree('NP', [('rate', 'NN')]), ('longer', 'RBR'), Tree('NP', [('period', 'NN')]), ('.', '.')]), Tree('S', [('nevertheless', 'RB'), (',', ','), ('said', 'VBD'), Tree('NP', [('brenda', 'NN')]), Tree('NP', [('malizia', 'NN')]), Tree('NP', [('negu', 'NN

In [66]:
# https://www.nltk.org/howto/chunk.html
# Le texte a déjà été chunk et pos tag
from nltk import tagstr2tree

gold_chunked_text = tagstr2tree(sentence_corpus[1])
print(gold_chunked_text)
gold_chunked_text.draw()
#[nltk.tag.str2tuple(t) for t in gold_chunked_text]
# print(tagged_token)

(S
  (NP longer/JJR maturities/NNS)
  are/VBP
  thought/VBN
  to/TO
  indicate/VB
  (NP declining/VBG interest_6/NN rates/NNS)
  because/IN
  (NP they/PP)
  permit/VBP
  (NP portfolio/NN managers/NNS)
  to/TO
  retain/VB
  relatively/RB
  (NP higher/JJR rates/NNS)
  for/IN
  (NP a/DT longer/JJR period/NN)
  ./.)


# Count Vectorizer

In [9]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
# ngram_range=(1,2) I might use this later, but I'm not too sure I need it right now
# TODO My vectorizers work as intended, but they are on the sentences, which are not stemmed. I need to fix this
vectorizer = CountVectorizer(input="content", analyzer='word')
X_count = vectorizer.fit_transform(sent_corpus_stemmed)

print(vectorizer.get_feature_names_out())
print(X_count.toarray())

['00' '000' '01' ... 'zinc' 'zoet' 'zurich']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# TF-IDF Vectorizer

In [10]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer
Tfvectorizer = TfidfVectorizer(input="content", analyzer='word')
X_tfidf = vectorizer.fit_transform(sent_corpus_stemmed)
print(vectorizer.get_feature_names_out())
print(X_tfidf.shape)
print(X_tfidf.toarray())

['00' '000' '01' ... 'zinc' 'zoet' 'zurich']
(2368, 5259)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# Stop words

In [ ]:
# https://realpython.com/nltk-nlp-python/#:~:text=Natural%20language%20processing%20(NLP)%20is,and%20contains%20human%2Dreadable%20text
# https://stackoverflow.com/questions/52712254/how-to-eliminate-stop-words-only-using-scikit-learn
stop_words = set(stopwords.words("english"))
filtered_list = [word for word in words_in_quote if word.casefold() not in stop_words]